# PET ROI SUV Statistics

This notebook will take in ROI files (NIfTI format) and calculate SUV statistics on the PET scan.

## Introduction

In this notebook, we will:
1. Load ROI files in NIfTI format.
2. Load the corresponding PET scan.
3. Calculate SUV statistics for the regions of interest.



## 1. Convert PET DICOM scans to NIFTI format

In [78]:
import dicom2nifti as d2n

PET_path = "/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/SUV-P8ct_BS_40-60_2.3IT_4it_PSFoff"
output_path = "/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/"

d2n.convert_directory(PET_path, output_path)

print(f'DICOM files from {PET_path} have been converted to NIfTI format and saved to {output_path}')

DICOM files from /Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/SUV-P8ct_BS_40-60_2.3IT_4it_PSFoff have been converted to NIfTI format and saved to /Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026


## 2. Load ROI files and convert PET scans to SUV values

In [59]:
import nibabel as nib
import numpy as np
import pydicom
import datetime
import os
from scipy import interpolate
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import IPython
from IPython.display import display, clear_output
from matplotlib.cm import ScalarMappable


# Upscale SUV values (256,256) to segmentation mask resolution (512,512)
def upscale_suv_values_3d(suv_values, new_shape):
    # Get the shape of the original SUV values array
    original_shape = suv_values.shape
    
    # Create arrays of coordinates for the original and new SUV values arrays
    x = np.linspace(0, 1, original_shape[0])
    y = np.linspace(0, 1, original_shape[1])
    z = np.linspace(0, 1, original_shape[2])
    
    new_x = np.linspace(0, 1, new_shape[0])
    new_y = np.linspace(0, 1, new_shape[1])
    new_z = np.linspace(0, 1, new_shape[2])
    
    # Create meshgrids of the coordinates
    x_mesh, y_mesh, z_mesh = np.meshgrid(x, y, z, indexing='ij')
    new_x_mesh, new_y_mesh, new_z_mesh = np.meshgrid(new_x, new_y, new_z, indexing='ij')
    
    # Interpolate the SUV values to the new coordinates in each dimension
    suv_interpolated = np.zeros(new_shape)
    for i in range(original_shape[2]):
        suv_interpolated[:, :, i] = interpolate.interpn((x, y), suv_values[:, :, i], (new_x_mesh[:, :, i], new_y_mesh[:, :, i]), method='linear', bounds_error=False, fill_value=None)
    
    return suv_interpolated

def load_nifti_file(filepath):
    """Load a NIfTI file and return the data array."""
    nifti_img = nib.load(filepath)
    data = nifti_img.get_fdata()
    return data

def convert_raw_PET_to_SUV(pet_dicom, pet_nifti):
    PET_data = load_nifti_file(pet_nifti)
    print(pet_dicom)
    full_path = os.path.join(pet_dicom, os.listdir(pet_dicom)[0])
    ds = pydicom.dcmread(full_path)
    
    # Get Scan time (Osirix uses SeriesTime, but can change to AcquisitionTime. Series time seems more precise)
    scantime = parse_time(ds.SeriesTime)
    # Start Time for the Radiopharmaceutical Injection
    injection_time = parse_time(ds.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime)
    # Half Life for Radionuclide # seconds
    half_life = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife) 
    # Total dose injected for Radionuclide
    injected_dose = float(ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
    
    # Calculate dose decay correction factor
    decay_correction_factor = np.exp(-np.log(2)*((scantime-injection_time).seconds)/half_life)

    
    patient_weight = ds.PatientWeight * 1000
    SUV_factor = (patient_weight) / (injected_dose * decay_correction_factor)
    
    return(PET_data * SUV_factor)

def calculate_suv_statistics(roi_data, pet_data):
    """Calculate SUV statistics for each unique ROI."""
    unique_rois = np.unique(roi_data)
    suv_stats = {}
    
    for roi in unique_rois:
        if roi == 0:
            continue  # Skip the background
        roi_mask = roi_data == roi
        suv_values = pet_data[roi_mask]
        
        mean_suv = np.mean(suv_values)
        median_suv = np.median(suv_values)
        std_suv = np.std(suv_values)
        
        suv_stats[roi] = {
            'mean': mean_suv,
            'median': median_suv,
            'std': std_suv
        }
    return suv_stats

# Function to parse time with or without fractional seconds
def parse_time(time_str):
    try:
        # Try parsing with fractional seconds
        return datetime.datetime.strptime(time_str, '%H%M%S.%f')
    except ValueError:
        # Fallback to parsing without fractional seconds
        return datetime.datetime.strptime(time_str, '%H%M%S')

def visualize_array(array):
    # Define a function to plot a heatmap of a given layer
    def plot_heatmap(layer):
        clear_output(wait=True)  # Clear the previous output without flashing
        fig, ax = plt.subplots(figsize=(8, 6))
        im = ax.imshow(array[:, :, layer], cmap='viridis', origin='lower')
        # Create a ScalarMappable for the colorbar
        sm = ScalarMappable(cmap='viridis')
        sm.set_array(array[:, :, layer])
        plt.colorbar(sm, ax=ax, label='Value')
        ax.set_title(f'Layer {layer}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        plt.show()

    # Use interact to create a slider for scrolling through layers
    interact(plot_heatmap, layer=IntSlider(min=0, max=array.shape[2] - 1, step=1, value=0))


In [ ]:
# Segmentation Map
total_segmentator_names = [
    "background",
    "spleen",
    "kidney_right",
    "kidney_left",
    "gallbladder",
    "liver",
    "stomach",
    "pancreas",
    "adrenal_gland_right",
    "adrenal_gland_left",
    "lung_upper_lobe_left",
    "lung_lower_lobe_left",
    "lung_upper_lobe_right",
    "lung_middle_lobe_right",
    "lung_lower_lobe_right",
    "esophagus",
    "trachea",
    "thyroid_gland",
    "small_bowel",
    "duodenum",
    "colon",
    "urinary_bladder",
    "prostate",
    "kidney_cyst_left",
    "kidney_cyst_right",
    "sacrum",
    "vertebrae_S1",
    "vertebrae_L5",
    "vertebrae_L4",
    "vertebrae_L3",
    "vertebrae_L2",
    "vertebrae_L1",
    "vertebrae_T12",
    "vertebrae_T11",
    "vertebrae_T10",
    "vertebrae_T9",
    "vertebrae_T8",
    "vertebrae_T7",
    "vertebrae_T6",
    "vertebrae_T5",
    "vertebrae_T4",
    "vertebrae_T3",
    "vertebrae_T2",
    "vertebrae_T1",
    "vertebrae_C7",
    "vertebrae_C6",
    "vertebrae_C5",
    "vertebrae_C4",
    "vertebrae_C3",
    "vertebrae_C2",
    "vertebrae_C1",
    "heart",
    "aorta",
    "pulmonary_vein",
    "brachiocephalic_trunk",
    "subclavian_artery_right",
    "subclavian_artery_left",
    "common_carotid_artery_right",
    "common_carotid_artery_left",
    "brachiocephalic_vein_left",
    "brachiocephalic_vein_right",
    "atrial_appendage_left",
    "superior_vena_cava",
    "inferior_vena_cava",
    "portal_vein_and_splenic_vein",
    "iliac_artery_left",
    "iliac_artery_right",
    "iliac_vena_left",
    "iliac_vena_right",
    "humerus_left",
    "humerus_right",
    "scapula_left",
    "scapula_right",
    "clavicula_left",
    "clavicula_right",
    "femur_left",
    "femur_right",
    "hip_left",
    "hip_right",
    "spinal_cord",
    "gluteus_maximus_left",
    "gluteus_maximus_right",
    "gluteus_medius_left",
    "gluteus_medius_right",
    "gluteus_minimus_left",
    "gluteus_minimus_right",
    "autochthon_left",
    "autochthon_right",
    "iliopsoas_left",
    "iliopsoas_right",
    "brain",
    "skull",
    "rib_left_1",
    "rib_left_2",
    "rib_left_3",
    "rib_left_4",
    "rib_left_5",
    "rib_left_6",
    "rib_left_7",
    "rib_left_8",
    "rib_left_9",
    "rib_left_10",
    "rib_left_11",
    "rib_left_12",
    "rib_right_1",
    "rib_right_2",
    "rib_right_3",
    "rib_right_4",
    "rib_right_5",
    "rib_right_6",
    "rib_right_7",
    "rib_right_8",
    "rib_right_9",
    "rib_right_10",
    "rib_right_11",
    "rib_right_12",
    "sternum",
    "costal_cartilages"
]

In [5]:
nifti_path = "/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/302_suv-p8ct_bs_40-60_23it_4it_psfoff.nii.gz"
dicom_path = "/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/SUV-P8ct_BS_40-60_2.3IT_4it_PSFoff"

SUV_vals = convert_raw_PET_to_SUV(dicom_path, nifti_path)
SUV_vals.shape
array_range = np.ptp(SUV_vals)
print(array_range)

/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/SUV-P8ct_BS_40-60_2.3IT_4it_PSFoff
87.99077179560045


$$
\text{decay\_correction\_factor} = \exp\left(-\ln(2) \cdot \frac{\text{{scantime}} - \text{{injection\_time}}}{\text{{half\_life}}}\right)
$$


## 3. Calculate SUV statistics within ROIs

In [60]:
# Jupyter magic command to display plots inline
%matplotlib nbagg
visualize_array(SUV_vals)


interactive(children=(IntSlider(value=0, description='layer', max=827), Output()), _dom_classes=('widget-inter…

In [91]:
segmentation_path = "/Users/williamlee/Documents/UC Davis COVID Study/COVID Patients/1697954_FDG_COVID_Pt002_JP/20121026/segmentation1.nii"
segmentation_img = nib.load(segmentation_path)
segmentation_data = segmentation_img.get_fdata()

new_shape = (512, 512, segmentation_data.shape[2])

upscaled_suv_values = upscale_suv_values_3d(SUV_vals, new_shape)
print(upscaled_suv_values.shape)

stats = calculate_suv_statistics(segmentation_data, upscaled_suv_values)

# Convert array to a pandas DataFrame for better display
df = pd.DataFrame(stats)

# Display the DataFrame
display(df)

(512, 512, 828)


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0
mean,0.816201,2.218232,0.770853,3.375697,1.276223,2.995071,2.635592,1.579427,1.661155,0.686801,...,0.676592,0.686799,0.936173,0.940037,0.650369,0.433225,0.463777,0.336334,1.265180,1.456310
median,0.793080,1.763944,0.705256,3.562352,1.100223,2.671892,2.826495,1.538411,1.606011,0.665961,...,0.486785,0.628322,0.762420,0.695548,0.516829,0.385168,0.359747,0.298564,0.799522,0.983471
std,0.363481,1.278214,0.426966,0.624017,0.761872,1.931598,1.061141,0.270805,0.404776,0.483304,...,0.592306,0.497242,0.939885,1.045145,0.700056,0.250183,0.273606,0.151981,1.189634,1.234656


In [82]:
segmentation_data.shape

(512, 512, 828)

In [83]:
SUV_vals.shape

(256, 256, 828)